In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

In [ ]:
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells.h5ad')

In [ ]:
adata = sc.read(data_dir_NHDP + 'whole_cohort1_processed.h5ad')

In [ ]:
sc.pl.umap(adata, color = ['cell_type'])

In [ ]:
adata_M = adata[adata.obs['cell_type'] == 'Fibroblast', :]

In [ ]:
adata_M

In [ ]:
if 'CXCR2' in adata_M.var_names:
    print('yes')

In [ ]:
adata_T = adata_T.raw.to_adata()

In [ ]:
adata_M = adata_M.raw.to_adata()

In [ ]:
lrpair = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/lr_network_unique.tsv', sep='\t')
# get the lr pairs including ligand(from) and receptor(to) from the lrpair file with database as 'kegg'
lrpair_kegg = lrpair[lrpair['database'] == 'kegg']
lrpair_kegg = lrpair_kegg[['from', 'to']]
# change from as L and to as R
lrpair_kegg.columns = ['L', 'R']
lrpair_kegg

## CD4 EX

In [ ]:
adata_CD4EX = adata_T[adata_T.obs['leiden'] == '3', :]
# adata_M = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_M_cells.h5ad')


In [ ]:
sc.pl.umap(adata_M, color='leiden', legend_loc='on data')

In [ ]:

adata_M_pseudo = scRNA2PseudoBulkAnnData(adata_M, sample_id_col='sample_id')
adata_T_pseudo = scRNA2PseudoBulkAnnData(adata_CD4EX, sample_id_col='sample_id')

In [ ]:
adata_M.raw = adata_M
adata_CD4EX.raw = adata_CD4EX

In [ ]:
DEG_1 = pd.read_csv(data_dir_NHDP + 'CD4EX_DEGs.csv')
DEG_2 = paird_ttest(adata_M, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2.to_csv(data_dir_NHDP + 'Fibro_DEGs.csv')
DEG_2 = pd.read_csv(data_dir_NHDP + 'Fibro_DEGs.csv')

In [ ]:
DEG_2

### CD4 vs M

In [ ]:
adata_M_pseudo = adata_M_pseudo[adata_M_pseudo.obs['sample_id'].isin(adata_T_pseudo.obs['sample_id'])]
adata_T_pseudo = adata_T_pseudo[adata_T_pseudo.obs['sample_id'].isin(adata_M_pseudo.obs['sample_id'])]

In [ ]:
DEG_2.columns

In [ ]:
#name first column as gene
DEG_1 = DEG_1.rename(columns = {'Unnamed: 0':'gene'})
DEG_2 = DEG_2.rename(columns = {'Unnamed: 0':'gene'})
# filter DEGs by pval < 0.01
DEG_1 = DEG_1[DEG_1['pval'] < 0.01]
DEG_2 = DEG_2[DEG_2['pval'] < 0.01]
gene_T = DEG_1['gene'].tolist()
gene_M = DEG_2['gene'].tolist()
gene_T_matrix = adata_T_pseudo[:, gene_T].X
gene_M_matrix = adata_M_pseudo[:, gene_M].X
gene_T_df = pd.DataFrame(gene_T_matrix, columns=gene_T, index=adata_T_pseudo.obs['sample_id'])
gene_M_df = pd.DataFrame(gene_M_matrix, columns=gene_M, index=adata_M_pseudo.obs['sample_id'])
gene_T_df.columns = [i + '_T' for i in gene_T_df.columns]
gene_M_df.columns = [i + '_M' for i in gene_M_df.columns]
gene_df = gene_T_df
gene_df = pd.merge(gene_df, gene_M_df, on='sample_id')

In [ ]:
print(DEG_1)
print(DEG_2)

In [ ]:
gene_df.columns = gene_df.columns.str.replace('_x', '')
gene_df

In [ ]:
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')
gene_df['treatment'].value_counts()

In [ ]:
# remove the patient id with only one sample
patient_counts = gene_df['patient_id'].value_counts()
single_patient = patient_counts[patient_counts == 1].index.tolist()
gene_df = gene_df[~gene_df['patient_id'].isin(single_patient)]
gene_df['treatment'].value_counts()

In [ ]:
adata_T_pseudo = adata_T_pseudo[adata_T_pseudo.obs['sample_id'].isin(gene_df.index)]
adata_M_pseudo = adata_M_pseudo[adata_M_pseudo.obs['sample_id'].isin(gene_df.index)]
adata_T_pseudo.obs['timepoint'].value_counts()

In [ ]:
# export the gene_df to csv
gene_df.to_csv(data_dir_NHDP + 'gene_df_CD4EX_Fibro.csv')

In [ ]:
DEG_pairs = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/IV_regression_results_CD4EXvsFibro.xlsx')
# drop the p_value column and r_squared column
DEG_pairs = DEG_pairs.drop(columns=['p_value', 'r_squared'])
# set the first two columns as g1 and g2 in DEG_pairs
DEG_pairs = DEG_pairs.rename(columns = {'gene_T':'g1', 'gene_cell':'g2'})

In [ ]:
DEG_pairs

In [ ]:
# calculate the correlation between in the DEG_pairs
from scipy.stats import spearmanr 
DEG_pairs['g1vsg2_correlation'] = 0

for index, row in DEG_pairs.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    g1 = g1.replace('_T', '')
    g2 = g2.replace('_M', '')
    # use the adata_T_pseudo and adata_M_pseudo to get the g1 and g2
    if g1 not in adata_T_pseudo.var_names:
        continue
    g1_T = adata_T_pseudo[:, g1].X
    g1_T = pd.DataFrame(g1_T, columns=[g1], index=adata_T_pseudo.obs['sample_id'])
    g1_T = g1_T.dropna()
    if g2 not in adata_M_pseudo.var_names:
        continue
    g2_M = adata_M_pseudo[:, g2].X
    g2_M = pd.DataFrame(g2_M, columns=[g2], index=adata_M_pseudo.obs['sample_id'])
    g2_M = g2_M.dropna()
    # make a dataframe g1, g2 and L*R
    corr = spearmanr(g1_T, g2_M)[0]
    DEG_pairs.loc[index, 'g1vsg2_correlation'] = corr
print(DEG_pairs)


In [ ]:
DEG_pairs = DEG_pairs[DEG_pairs['g1vsg2_correlation'] > 0.4]
# sort the DEG_pairs by g1vsg2_correlation
DEG_pairs = DEG_pairs.sort_values(by='g1vsg2_correlation', ascending=False)
DEG_pairs

In [ ]:
for index, row in DEG_pairs.iterrows():
    g1 = row['g1']
    g2 = row['g2']

In [ ]:
lrpair_kegg

In [ ]:
from causallearn.utils.cit import CIT
results = pd.DataFrame(columns=['g1', 'g2', 'L', 'R', 'pValue'])

for index, row in DEG_pairs.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    if g2.endswith('_M'):
        for L, R in lrpair_kegg.values:
            # check if L and R can be found in the DEG expression data
            if L in adata_T_pseudo.var_names and R  in adata_M_pseudo.var_names:
                # get the L_T and R_B from pseudo_T and pseudo_M
                L_T = adata_T_pseudo[:, L].X
                L_T = pd.DataFrame(L_T, columns=[L], index=adata_T_pseudo.obs['sample_id'])
                L_T = L_T.dropna()
                R_M = adata_M_pseudo[:, R].X
                R_M = pd.DataFrame(R_M, columns=[R], index=adata_M_pseudo.obs['sample_id'])
                R_M = R_M.dropna()
                # calculate the L*R
                # Ensure the column names match
                L_T.columns = [R+'_vs_'+L]
                R_M.columns = L_T.columns

                # perform the multiplication and change L_R to 1-dimensional
                L_R = L_T.multiply(R_M, axis=0)
                L_R = L_R.iloc[:,0]

                # make a dataframe g1, g2 and L*R
                if g1 not in gene_df.columns or g2 not in gene_df.columns:
                    continue
                
                g1_T = gene_df[g1]
                g2_M = gene_df[g2]
                df = pd.DataFrame({g1: g1_T, g2: g2_M, 'L*R': L_R})
                # remove the rows with NA
                df = df.dropna()

                # convert df to numpy array
                df_numpy = df.to_numpy()

                fisherz_obj = CIT(df_numpy, "fisherz")
                pValue = fisherz_obj( 0, 1, [2] )
                if pValue > 0.1:
                     results = pd.concat([results, pd.DataFrame({'g1': [g1], 'g2': [g2], 'L': [L], 'R': [R], 'pValue': [pValue]})], ignore_index=True)
                results = pd.concat([results, pd.DataFrame({'g1': [g1], 'g2': [g2], 'L': [L], 'R': [R], 'pValue': [pValue]})], ignore_index=True)
                
print(results)

In [ ]:
adata_M_pseudo

In [ ]:
adata

In [ ]:
# remove the columns except g1, g2, L, R and pValue
results = results[results['pValue']>0.1]

In [ ]:
results

In [ ]:
#calculate the correlation between g1 and g2
from scipy.stats import spearmanr 
results['g1vsg2_correlation'] = 0
for index, row in results.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    g1 = g1.replace('_T', '')
    g2 = g2.replace('_M', '')
    # use the adata_T_pseudo and adata_M_pseudo to get the g1 and g2
    g1_T = adata_T_pseudo[:, g1].X
    g1_T = pd.DataFrame(g1_T, columns=[g1], index=adata_T_pseudo.obs['sample_id'])
    g1_T = g1_T.dropna()
    g2_M = adata_M_pseudo[:, g2].X
    g2_M = pd.DataFrame(g2_M, columns=[g2], index=adata_M_pseudo.obs['sample_id'])
    g2_M = g2_M.dropna()
    # make a dataframe g1, g2 and L*R
    corr = spearmanr(g1_T, g2_M)[0]
    results.loc[index, 'g1vsg2_correlation'] = corr
print(results)

In [ ]:
# claculate the correlation between g1 and L
results['g1vsL_correlation'] = 0   
for index, row in results.iterrows():
    g1 = row['g1']
    L = row['L']
    g1 = g1.replace('_T', '')
    # use the adata_T_pseudo and adata_M_pseudo to get the g1 and g2
    g1_T = adata_T_pseudo[:, g1].X
    g1_T = pd.DataFrame(g1_T, columns=[g1], index=adata_T_pseudo.obs['sample_id'])
    g1_T = g1_T.dropna()
    L_T = adata_T_pseudo[:, L].X
    L_T = pd.DataFrame(L_T, columns=[L], index=adata_T_pseudo.obs['sample_id'])
    L_T = L_T.dropna()
    # make a dataframe g1, g2 and L*R
    corr = spearmanr(g1_T, L_T)[0]
    results.loc[index, 'g1vsL_correlation'] = corr

# claculate the correlation between g2 and R
results['g2vsR_correlation'] = 0
for index, row in results.iterrows():
    g2 = row['g2']
    R = row['R']
    g2 = g2.replace('_M', '')
    # use the adata_T_pseudo and adata_M_pseudo to get the g1 and g2
    g2_M = adata_M_pseudo[:, g2].X
    g2_M = pd.DataFrame(g2_M, columns=[g2], index=adata_M_pseudo.obs['sample_id'])
    g2_M = g2_M.dropna()
    R_M = adata_M_pseudo[:, R].X
    R_M = pd.DataFrame(R_M, columns=[R], index=adata_M_pseudo.obs['sample_id'])
    R_M = R_M.dropna()
    # make a dataframe g1, g2 and L*R
    corr = spearmanr(g2_M, R_M)[0]
    results.loc[index, 'g2vsR_correlation'] = corr

In [ ]:
# calculate the correlation between g1 and L*R
results['g1vsLR_correlation'] = 0
for index, row in results.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    L = row['L']
    R = row['R']
    g1 = g1.replace('_T', '')
    g2 = g2.replace('_M', '')
    # use the adata_T_pseudo and adata_M_pseudo to get the g1 and g2
    g1_T = adata_T_pseudo[:, g1].X
    g1_T = pd.DataFrame(g1_T, columns=[g1], index=adata_T_pseudo.obs['sample_id'])
    g1_T = g1_T.dropna()
    g2_M = adata_M_pseudo[:, g2].X
    g2_M = pd.DataFrame(g2_M, columns=[g2], index=adata_M_pseudo.obs['sample_id'])
    g2_M = g2_M.dropna()
    L_T = adata_T_pseudo[:, L].X
    L_T = pd.DataFrame(L_T, columns=[L], index=adata_T_pseudo.obs['sample_id'])
    L_T = L_T.dropna()
    R_M = adata_M_pseudo[:, R].X
    R_M = pd.DataFrame(R_M, columns=[R], index=adata_M_pseudo.obs['sample_id'])
    R_M = R_M.dropna()
    # calculate the L*R
    # Ensure the column names match
    L_T.columns = [R+'_vs_'+L]
    R_M.columns = L_T.columns

    # perform the multiplication and change L_R to 1-dimensional
    L_R = L_T.multiply(R_M, axis=0)
    L_R = L_R.iloc[:,0]
    # make a dataframe g1, g2 and L*R
    corr = spearmanr(g1_T, L_R)[0]
    results.loc[index, 'g1vsLR_correlation'] = corr

# calculate the correlation between g2 and L*R
results['g2vsLR_correlation'] = 0
for index, row in results.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    L = row['L']
    R = row['R']
    g1 = g1.replace('_T', '')
    g2 = g2.replace('_M', '')
    # use the adata_T_pseudo and adata_M_pseudo to get the g1 and g2
    g1_T = adata_T_pseudo[:, g1].X
    g1_T = pd.DataFrame(g1_T, columns=[g1], index=adata_T_pseudo.obs['sample_id'])
    g1_T = g1_T.dropna()
    g2_M = adata_M_pseudo[:, g2].X
    g2_M = pd.DataFrame(g2_M, columns=[g2], index=adata_M_pseudo.obs['sample_id'])
    g2_M = g2_M.dropna()
    L_T = adata_T_pseudo[:, L].X
    L_T = pd.DataFrame(L_T, columns=[L], index=adata_T_pseudo.obs['sample_id'])
    L_T = L_T.dropna()
    R_M = adata_M_pseudo[:, R].X
    R_M = pd.DataFrame(R_M, columns=[R], index=adata_M_pseudo.obs['sample_id'])
    R_M = R_M.dropna()
    # calculate the L*R
    # Ensure the column names match
    L_T.columns = [R+'_vs_'+L]
    R_M.columns = L_T.columns

    # perform the multiplication and change L_R to 1-dimensional
    L_R = L_T.multiply(R_M, axis=0)
    L_R = L_R.iloc[:,0]
    # make a dataframe g1, g2 and L*R
    corr = spearmanr(g2_M, L_R)[0]
    results.loc[index, 'g2vsLR_correlation'] = corr


In [ ]:
results

In [ ]:
from scipy.stats import spearmanr
import statsmodels.api as sm
import pandas as pd

# Initialize the correlation column in results DataFrame
results['g1_residualvsg2_residuals_correlation'] = 0

for index, row in results.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    L = row['L']
    R = row['R']
    g1 = g1.replace('_T', '')
    g2 = g2.replace('_M', '')

    # Extract the expression data from adata_T_pseudo and adata_M_pseudo
    g1_T = adata_T_pseudo[:, g1].X.toarray().flatten()
    g2_M = adata_M_pseudo[:, g2].X.toarray().flatten()
    L_T = adata_T_pseudo[:, L].X.toarray().flatten()
    R_M = adata_M_pseudo[:, R].X.toarray().flatten()

    # Ensure the column names match for L and R
    L_R = L_T * R_M

    # Create DataFrames with g1, g2, and L*R
    df_g1 = pd.DataFrame({g1: g1_T, 'L*R': L_R})
    df_g2 = pd.DataFrame({g2: g2_M, 'L*R': L_R})
    df_g1 = df_g1.dropna()
    df_g2 = df_g2.dropna()

    # Perform the regression of g1 on L*R to get residuals
    X_g1 = sm.add_constant(df_g1['L*R'])  # Add a constant term to the predictor
    y_g1 = df_g1[g1]
    model_g1 = sm.OLS(y_g1, X_g1).fit()
    residuals_g1 = model_g1.resid

    # Perform the regression of g2 on L*R to get residuals
    X_g2 = sm.add_constant(df_g2['L*R'])  # Add a constant term to the predictor
    y_g2 = df_g2[g2]
    model_g2 = sm.OLS(y_g2, X_g2).fit()
    residuals_g2 = model_g2.resid

    # Calculate the correlation between the residuals of g1 and g2
    corr = spearmanr(residuals_g1, residuals_g2)[0]

    # Update the results DataFrame with the correlation
    results.at[index, 'g1_residualvsg2_residuals_correlation'] = corr

results


In [ ]:
from scipy.stats import spearmanr
import statsmodels.api as sm

# Initialize the correlation column in results DataFrame
results['g1vsresiduals_correlation'] = 0

for index, row in results.iterrows():
    g1 = row['g1']
    g2 = row['g2']
    L = row['L']
    R = row['R']
    g1 = g1.replace('_T', '')
    g2 = g2.replace('_M', '')

    # Extract the expression data from adata_T_pseudo and adata_M_pseudo
    g1_T = adata_T_pseudo[:, g1].X.toarray().flatten()
    g2_M = adata_M_pseudo[:, g2].X.toarray().flatten()
    L_T = adata_T_pseudo[:, L].X.toarray().flatten()
    R_M = adata_M_pseudo[:, R].X.toarray().flatten()

    # Ensure the column names match for L and R
    L_R = L_T * R_M

    # Create a DataFrame with g1, g2, and L*R
    df = pd.DataFrame({g1: g1_T, g2: g2_M, 'L*R': L_R})
    df = df.dropna()

    # Perform the regression of g2 on L*R to get residuals
    X = sm.add_constant(df['L*R'])  # Add a constant term to the predictor
    y = df[g2]
    model = sm.OLS(y, X).fit()
    residuals = model.resid

    # Calculate the correlation between g1 and residuals
    corr = spearmanr(df[g1], residuals)[0]

    # Update the results DataFrame with the correlation
    results.at[index, 'g1vsresiduals_correlation'] = corr

results


In [ ]:
results.to_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_CD4EXvsFirbo.xlsx')

In [ ]:
# change all the names with _M to _Fibro
results['g2'] = results['g2'].str.replace('_M', '_Fibro')


In [ ]:
# read in results
results = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_CD4EXvsM.xlsx')

#### draw heatmap

In [ ]:
top_genes_M = [
    'RNF144B',
    'SLC1A3',
    
    'TNFAIP3',
    'CH25H',
    'FKBP5',
    'CLN8',
    'GRASP',
    'SMIM3',
    'DUSP6',
    'HIST1H4H'
]
top_genes_T = [
    'CH25H',
    'TSC22D3',
    'TNFAIP3',
    'BIRC3',
    'SPON2',
    'ZFP36',
    'SLA',
    'ZFP36',
    'TXNIP',
    'IER2',
    'KLF10'
]

In [ ]:
import seaborn as sns
import scipy.stats as stats
from scipy.stats import spearmanr 
# Remove the sample_id that are not in the adata_T_pseudo
adata_M_pseudo = adata_M_pseudo[adata_M_pseudo.obs['sample_id'].isin(adata_T_pseudo.obs['sample_id'])]
adata_T_pseudo = adata_T_pseudo[adata_T_pseudo.obs['sample_id'].isin(adata_M_pseudo.obs['sample_id'])]

# Initialize an empty matrix to store the correlation values
corr_matrix = np.zeros((len(top_genes_T), len(top_genes_M)))
pval_matrix = np.zeros((len(top_genes_T), len(top_genes_M)))

# Calculate the correlation between the top genes of T and M cells
for i, gene_T in enumerate(top_genes_T):
    for j, gene_M in enumerate(top_genes_M):
        # remove the '_T' and '_M' from the gene names
        gene_T = gene_T.replace('_T', '')
        gene_M = gene_M.replace('_M', '')
        # Get the expression data of the two genes and flatten them to 1D arrays
        gene_T_expression = adata_T_pseudo[:, adata_T_pseudo.var_names == gene_T].X.flatten()
        gene_M_expression = adata_M_pseudo[:, adata_M_pseudo.var_names == gene_M].X.flatten()
        
        # Calculate the correlation
        corr, pval = spearmanr(gene_T_expression, gene_M_expression)
        
        # Store the correlation value in the matrix
        corr_matrix[i, j] = corr
        pval_matrix[i, j] = -np.log10(pval)


# Plot the correlation matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1,
            xticklabels=top_genes_M, yticklabels=top_genes_T)
plt.title('Correlation between Highly Correlated Genes of CD4EX and M')
plt.xlabel('M Genes')
plt.ylabel('CD4EX Genes')
plt.show()

In [ ]:

# Plotting the heatmap with correlation values
plt.figure(figsize=(12, 10))
heatmap = sns.heatmap(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1,
                      xticklabels=top_genes_M, yticklabels=top_genes_T,
                      annot=False)  # Turn off default annotation

# Overlaying the correlation values and -log10 of p-values
for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[1]):
        plt.text(j + 0.5, i + 0.5, f'{corr_matrix[i, j]:.2f}\n({pval_matrix[i, j]:.2f})',
                 horizontalalignment='center', verticalalignment='center', fontsize=11, color='black' if abs(corr_matrix[i, j]) > 0.5 else 'black')

plt.title('Correlation between Top DEGs of CD4EX and M')
plt.xlabel('M Genes')
plt.ylabel('CD4EX Genes')
plt.show()

In [ ]:
colors = ["grey", "blue"]  # Start with grey and end with blue
cmap = mcolors.LinearSegmentedColormap.from_list("grey_to_blue", colors)


In [ ]:
sc.pl.umap(adata_CD4EX, color = ['SPON2', 'SELL'], color_map = cmap)

In [ ]:
sc.pl.umap(adata_M, color = ['FKBP5', 'MADCAM1'], color_map = cmap)

In [ ]:
sc.pl.umap(adata_CD4EX, color = 'timepoint')
sc.pl.umap(adata_M, color = 'timepoint')

#### plot the scatter plots


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import spearmanr

def plot_CIT_DEGcorr(g1, g2, L, R, DEG_exp, pseudo_1, pseudo_2):
    # Define a custom palette
    palette = {'pre': "#E69F00", 'on': "#56B4E9"}
    #replace the treatment column 0 pre and 1 on
    gene_df['treatment'] = gene_df['treatment'].replace({0: 'pre', 1: 'on'})
    # Set font sizes
    title_fontsize = 14
    axis_fontsize = 14
    legend_fontsize = 12

    def adjust_plot_limits(ax, x_data, y_data):
        x_min, x_max = x_data.min(), x_data.max()
        y_min, y_max = y_data.min(), y_data.max()
        x_padding = (x_max - x_min) * 0.1
        y_padding = (y_max - y_min) * 0.1
        ax.set_xlim(x_min - x_padding, x_max + x_padding)
        ax.set_ylim(y_min - y_padding, y_max + y_padding)

    # Plot the jointplot between DEG of g1 and g2
    g1_exp = DEG_exp[g1]
    g2_exp = DEG_exp[g2]
    df1 = pd.DataFrame({g1: g1_exp, g2: g2_exp, 'treatment': DEG_exp['treatment']})
    df1 = df1.dropna()
    g = sns.jointplot(x=g1, y=g2, data=df1, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    sns.regplot(x=g1, y=g2, data=df1, scatter=False, ax=g.ax_joint, color='black')
    adjust_plot_limits(g.ax_joint, df1[g1], df1[g2])
    
    g.ax_joint.tick_params(left=False, bottom=False)
    g.ax_marg_x.tick_params(bottom=False)
    g.ax_marg_y.tick_params(left=False)
    cor = spearmanr(df1[g1], df1[g2])[0]
    plt.suptitle(f'Correlation between {g1} and {g2} is {cor:.3f}', fontsize=title_fontsize)
    plt.subplots_adjust(top=0.95)
    plt.show()

     # Plot the jointplot between DEG of g1 and g2 residue
    L_values = pseudo_1[:, L].X.toarray().flatten()
    R_values = pseudo_2[:, R].X.toarray().flatten()
    L_R = L_values * R_values
    X = sm.add_constant(L_R)

    # Fit OLS model for g2
    model_g2 = sm.OLS(g2_exp, X).fit()
    residuals_g2 = model_g2.resid

    # Fit OLS model for g1
    model_g1 = sm.OLS(g1_exp, X).fit()
    residuals_g1 = model_g1.resid

    #   Create DataFrame for residuals
    df_res = pd.DataFrame({
        'G_1_residual': residuals_g1,
        'G_2_residual': residuals_g2,
        'treatment': DEG_exp['treatment']
    })

    # Plot the residuals
    g = sns.jointplot(x='G_1_residual', y='G_2_residual', data=df_res, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    sns.regplot(x='G_1_residual', y='G_2_residual', data=df_res, scatter=False, ax=g.ax_joint, color='black')

    # Adjust plot limits if needed
    adjust_plot_limits(g.ax_joint, df_res['G_1_residual'], df_res['G_2_residual'])

    # Customize plot appearance
    g.ax_joint.tick_params(left=False, bottom=False)
    g.ax_marg_x.tick_params(bottom=False)
    g.ax_marg_y.tick_params(left=False)

    # Calculate and display correlation
    corr_res = spearmanr(df_res['G_1_residual'], df_res['G_2_residual'])[0]
    plt.suptitle(f'Correlation between residuals of G_1 and G_2 is {corr_res:.3f}', fontsize=title_fontsize)
    plt.subplots_adjust(top=0.95)
    plt.show()


    # Plot the jointplot between DEG of g1 and L
    L_exp = pseudo_1[:, L].X.toarray().flatten()
    L_exp = pd.DataFrame(L_exp, columns=[L], index=pseudo_1.obs['sample_id'])
    df2 = pd.DataFrame({g1: g1_exp, L: L_exp.iloc[:, 0], 'treatment': DEG_exp['treatment']})
    df2 = df2.dropna()
    g = sns.jointplot(x=g1, y=L, data=df2, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    sns.regplot(x=g1, y=L, data=df2, scatter=False, ax=g.ax_joint, color='black')
    adjust_plot_limits(g.ax_joint, df2[g1], df2[L])
    g.ax_joint.tick_params(left=False, bottom=False)
    g.ax_marg_x.tick_params(bottom=False)
    g.ax_marg_y.tick_params(left=False)
    corr2 = spearmanr(df2[g1], df2[L])[0]
    plt.suptitle(f'Correlation between {g1} and {L} is {corr2:.3f}', fontsize=title_fontsize)
    plt.subplots_adjust(top=0.95)
    plt.show()

    # Plot the jointplot between DEG of R and g2
    R_exp = pseudo_2[:, R].X.toarray().flatten()
    R_exp = pd.DataFrame(R_exp, columns=[R], index=pseudo_2.obs['sample_id'])
    df4 = pd.DataFrame({R: R_exp.iloc[:, 0], g2: g2_exp, 'treatment': DEG_exp['treatment']})
    df4 = df4.dropna()
    g = sns.jointplot(x=R, y=g2, data=df4, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    sns.regplot(x=R, y=g2, data=df4, scatter=False, ax=g.ax_joint, color='black')
    adjust_plot_limits(g.ax_joint, df4[R], df4[g2])
    g.ax_joint.tick_params(left=False, bottom=False)
    g.ax_marg_x.tick_params(bottom=False)
    g.ax_marg_y.tick_params(left=False)
    corr4 = spearmanr(df4[R], df4[g2])[0]
    plt.suptitle(f'Correlation between {R} and {g2} is {corr4:.3f}', fontsize=title_fontsize)
    plt.subplots_adjust(top=0.95)
    plt.show()

    # # Plot the jointplot between DEG of g1 and g2 residue
    # L_values = pseudo_1[:, L].X.toarray().flatten()
    # R_values = pseudo_2[:, R].X.toarray().flatten()
    # L_R = L_values * R_values
    # X = sm.add_constant(L_R)
    # model = sm.OLS(g2_exp, X).fit()
    # residue = model.resid
    # df_res = pd.DataFrame({g1: g1_exp, 'residue': residue, 'treatment': DEG_exp['treatment']})
    # g = sns.jointplot(x=g1, y='residue', data=df_res, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    # sns.regplot(x=g1, y='residue', data=df_res, scatter=False, ax=g.ax_joint, color='black')
    # adjust_plot_limits(g.ax_joint, df_res[g1], df_res['residue'])
    # g.ax_joint.tick_params(left=False, bottom=False)
    # g.ax_marg_x.tick_params(bottom=False)
    # g.ax_marg_y.tick_params(left=False)
    # corr_res = spearmanr(df_res[g1], df_res['residue'])[0]
    # plt.suptitle(f'Correlation between {g1} and residue of {g2} is {corr_res:.3f}', fontsize=title_fontsize)
    # plt.subplots_adjust(top=0.95)
    # plt.show()

    # Plot the jointplot between g1 and L*R
    df3 = pd.DataFrame({g1: g1_exp, L + '*' + R: L_R, 'treatment': DEG_exp['treatment']})
    df3 = df3.dropna()
     # remove the sample with largest number for L*R
    # df3 = df3[df3[L + '*' + R] != df3[L + '*' + R].max()]
    g = sns.jointplot(x=g1, y=L + '*' + R, data=df3, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    sns.regplot(x=g1, y=L + '*' + R, data=df3, scatter=False, ax=g.ax_joint, color='black')
    adjust_plot_limits(g.ax_joint, df3[g1], df3[L + '*' + R])
    g.ax_joint.tick_params(left=False, bottom=False)
    g.ax_marg_x.tick_params(bottom=False)
    g.ax_marg_y.tick_params(left=False)
    corr3 = spearmanr(df3[g1], df3[L + '*' + R])[0]
    plt.suptitle(f'Correlation between {g1} and {L}*{R} is {corr3:.3f}', fontsize=title_fontsize)
    plt.subplots_adjust(top=0.95)
    plt.show()

    # Plot the jointplot between g2 and L*R
    df5 = pd.DataFrame({g2: g2_exp, L + '*' + R: L_R, 'treatment': DEG_exp['treatment']})
    df5 = df5.dropna()
    # df5 = df5[df5[L + '*' + R] != df5[L + '*' + R].max()]
    g = sns.jointplot(x=L + '*' + R, y=g2, data=df5, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    sns.regplot(x=L + '*' + R, y=g2, data=df5, scatter=False, ax=g.ax_joint, color='black')
    adjust_plot_limits(g.ax_joint, df5[L + '*' + R], df5[g2])
    # remove the sample with largest number for L*R
    g.ax_joint.tick_params(left=False, bottom=False)
    g.ax_marg_x.tick_params(bottom=False)
    g.ax_marg_y.tick_params(left=False)
    corr5 = spearmanr(df5[g2], df5[L + '*' + R])[0]
    plt.suptitle(f'Correlation between {g2} and {L}*{R} is {corr5:.3f}', fontsize=title_fontsize)
    plt.subplots_adjust(top=0.95)
    plt.show()


   



In [ ]:
plot_CIT_DEGcorr('CH25H_T', 'SMIM3_M', 'MPZ', 'MPZ', gene_df, adata_T_pseudo, adata_M_pseudo)